In [38]:
!pip install keras-tuner -q


[notice] A new release of pip is available: 23.0.1 -> 23.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [39]:
from tensorflow import keras
from tensorflow.keras import layers

# Hyper Parameter Optimization
Keras Tuner allows you to replace hardcoded hyper-parameters with a *search space;* a range of possible hyperparameter choices.  

To specify a *search space*, you'll define a model building function that takes hyperparameter (hp) ranges of values.

In [40]:
def build_model(hp):
    units = hp.Int('units', min_value=16, max_value=64, step=32)
    model = keras.Sequential([
        layers.Dense(units, activation='relu'),
        layers.Dense(10, activation='softmax')
    ])
    optimizer = hp.Choice('optimizer', values=['rmsprop','adam', 'sgd'])
    model.compile(optimizer=optimizer, loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


If you wish to adopt a more modular and configurable approach, you can also extend the HyperModle class and override the build method. This is useful if you want to use the same model architecture for different problems, or if you want to use the same model architecture with different hyperparameters.

In [41]:
import keras_tuner as kt

class SimpleMLP(kt.HyperModel):
    def __init__(self, input_shape, num_classes):
        self.input_shape = input_shape
        self.num_classes = num_classes

    def build(self, hp):
        units = hp.Int('units', min_value=16, max_value=64, step=32)
        model = keras.Sequential([
            layers.Dense(units, activation='relu'),
            layers.Dense(self.num_classes, activation='softmax')
        ])
        optimizer = hp.Choice('optimizer', values=['rmsprop','adam', 'sgd'])
        model.compile(optimizer=optimizer, loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])
        return model

hypermodel = SimpleMLP(input_shape=(784,), num_classes=10)

## Tuner
Next step is to define a *tuner* which is a for loop that will repeatedly...
> Pick a set of hyperparameter values
> Call the model-building function with these values to create a model
> Train the model and record its metrics

Several tuners are pre-built into kt such as *RandomSearch, BayesianOptimization, Hyperband.*

In [42]:
#build the tuner
tuner = kt.BayesianOptimization(build_model, objective='val_accuracy', max_trials=10, executions_per_trial=2, directory='mnist_kt_test', project_name='intro_to_kt_mnist', overwrite=True)

In [43]:
tuner.search_space_summary()

Search space summary
Default search space size: 2
units (Int)
{'default': None, 'conditions': [], 'min_value': 16, 'max_value': 64, 'step': 32, 'sampling': 'linear'}
optimizer (Choice)
{'default': 'rmsprop', 'conditions': [], 'values': ['rmsprop', 'adam', 'sgd'], 'ordered': False}


# Now that the MLP tuner's build and constraints are defined, load the dataset.

In [44]:
(x_train, y_train), (x_test, y_test) = keras.datasets.mnist.load_data()
x_train = x_train.reshape(-1, 28*28).astype('float32') / 255.0
x_test = x_test.reshape(-1, 28*28).astype('float32') / 255.0
x_train_full = x_train
y_train_full = y_train


In [45]:
num_val_samples = 10000
x_train, x_val = x_train_full[:-num_val_samples], x_train_full[-num_val_samples:]
y_train, y_val = y_train_full[:-num_val_samples], y_train_full[-num_val_samples:]


In [46]:
callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)]

tuner.search(x_train, y_train, epochs=50, validation_data=(x_val, y_val), callbacks=callbacks)

Trial 10 Complete [00h 01m 23s]
val_accuracy: 0.9712500274181366

Best val_accuracy So Far: 0.9736999869346619
Total elapsed time: 00h 20m 31s
INFO:tensorflow:Oracle triggered exit


In [47]:
top_n = 3
top_params = tuner.get_best_hyperparameters(num_trials=top_n)
top_models = tuner.get_best_models(num_models=top_n)

In [48]:
def get_best_epoch(hp):
    model=build_model(hp)
    callbacks = [keras.callbacks.EarlyStopping(monitor='val_loss', mode="min", patience=10)]
    history = model.fit(x_train, y_train, epochs=50, validation_data=(x_val, y_val), batch_size=128, callbacks=callbacks)
    val_loss_per_epoch = history.history['val_loss']
    best_epoch = val_loss_per_epoch.index(min(val_loss_per_epoch)) + 1
    print(f"Best epoch: {best_epoch}" )
    return best_epoch


In [49]:
def get_best_trained_model(hp):
    best_epoch = get_best_epoch(hp)
    #model = build_model(hp)
    model.fit(x_train, y_train, epochs=int(best_epoch*1.2), validation_data=(x_val, y_val), batch_size=128)
    return model

In [50]:
best_models=[]
for hp in top_params:
    model=get_best_trained_model(hp)
    model.evaluate(x_test, y_test)
    best_models.append(model)

Epoch 1/50
391/391 [==============================] - 1s 3ms/step - loss: 0.4562 - accuracy: 0.8780 - val_loss: 0.2462 - val_accuracy: 0.9307
Epoch 2/50
391/391 [==============================] - 1s 3ms/step - loss: 0.2357 - accuracy: 0.9328 - val_loss: 0.1989 - val_accuracy: 0.9458
Epoch 3/50
391/391 [==============================] - 1s 3ms/step - loss: 0.1864 - accuracy: 0.9456 - val_loss: 0.1659 - val_accuracy: 0.9544
Epoch 4/50
391/391 [==============================] - 1s 3ms/step - loss: 0.1553 - accuracy: 0.9553 - val_loss: 0.1456 - val_accuracy: 0.9594
Epoch 5/50
391/391 [==============================] - 1s 3ms/step - loss: 0.1339 - accuracy: 0.9617 - val_loss: 0.1372 - val_accuracy: 0.9623
Epoch 6/50
391/391 [==============================] - 1s 3ms/step - loss: 0.1174 - accuracy: 0.9662 - val_loss: 0.1285 - val_accuracy: 0.9644
Epoch 7/50
391/391 [==============================] - 1s 3ms/step - loss: 0.1047 - accuracy: 0.9700 - val_loss: 0.1239 - val_accuracy: 0.9644
Epoch 

NameError: name 'model' is not defined

In [ ]:
best_models = tuner.get_best_models(top_n)

In [ ]:
#show the best models


[<keras.engine.sequential.Sequential object at 0x0000028E81609288>, <keras.engine.sequential.Sequential object at 0x0000028E80266AC8>, <keras.engine.sequential.Sequential object at 0x0000028E80266C88>]


In [ ]:
tuner.results_summary()

Results summary
Results in mnist_kt_test\intro_to_kt_mnist
Showing 10 best trials
Objective(name="val_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
units: 48
optimizer: adam
Score: 0.9735999703407288

Trial 4 summary
Hyperparameters:
units: 48
optimizer: sgd
Score: 0.9690999984741211

Trial 0 summary
Hyperparameters:
units: 16
optimizer: rmsprop
Score: 0.9564999938011169

Trial 1 summary
Hyperparameters:
units: 16
optimizer: adam
Score: 0.9539999961853027

Trial 3 summary
Hyperparameters:
units: 16
optimizer: sgd
Score: 0.9538000226020813


In [ ]:
tuner.get_best_hyperparameters(num_trials=top_n)

In [ ]:
best_model = get_best_trained_model(top_params[0])

Epoch 1/50
391/391 [==============================] - 1s 2ms/step - loss: 0.4921 - accuracy: 0.8641 - val_loss: 0.2526 - val_accuracy: 0.9302
Epoch 2/50
391/391 [==============================] - 1s 2ms/step - loss: 0.2436 - accuracy: 0.9308 - val_loss: 0.2076 - val_accuracy: 0.9443
Epoch 3/50
391/391 [==============================] - 1s 1ms/step - loss: 0.1915 - accuracy: 0.9460 - val_loss: 0.1716 - val_accuracy: 0.9514
Epoch 4/50
391/391 [==============================] - 1s 2ms/step - loss: 0.1607 - accuracy: 0.9546 - val_loss: 0.1520 - val_accuracy: 0.9558
Epoch 5/50
391/391 [==============================] - 1s 1ms/step - loss: 0.1373 - accuracy: 0.9609 - val_loss: 0.1359 - val_accuracy: 0.9604
Epoch 6/50
391/391 [==============================] - 1s 2ms/step - loss: 0.1208 - accuracy: 0.9658 - val_loss: 0.1283 - val_accuracy: 0.9623
Epoch 7/50
391/391 [==============================] - 1s 2ms/step - loss: 0.1071 - accuracy: 0.9697 - val_loss: 0.1215 - val_accuracy: 0.9650
Epoch 

NameError: name 'model' is not defined